In [1]:
# Cell 1: Setup and Installations
# -------------------------------
# This cell installs all required Python libraries for the project.
!pip install pybind11 kaggle -q
print("pybind11 and kaggle insyalled")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 4.6 MB/s eta 0:00:00
pybind11 and kaggle insyalled


In [3]:
%%writefile pknumpy_shannon_v2.cpp
// Cell 2: Fixed C++ Backend (Optimized Shannon).

#include <pybind11/pybind11.h>
#include <pybind11/stl.h>
#include <pybind11/numpy.h>
#include <vector>
#include <numeric>
#include <cmath>
#include <algorithm>
#include <stdexcept>
#include <omp.h> // For Openmp

namespace py = pybind11;

// Loss Functions
py::array_t<double> sigmoid(py::array_t<double> x) {
    auto buf = x.request(); py::array_t<double> result(buf.size); auto res_buf = result.request();
    double *x_ptr = static_cast<double *>(buf.ptr), *res_ptr = static_cast<double *>(res_buf.ptr);
    #pragma omp parallel for
    for (ssize_t i=0; i<buf.size; ++i) {
        double val = std::max(-500.0, std::min(500.0, x_ptr[i]));
        res_ptr[i] = (val >= 0) ? (1.0 / (1.0 + std::exp(-val))) : (std::exp(val) / (1.0 + std::exp(val)));
    }
    std::vector<ssize_t> shape(x.shape(), x.shape() + x.ndim());
    return result.reshape(shape);
}

py::array_t<double> gradient(py::array_t<double> y_true, py::array_t<double> y_pred) {
    auto y_true_buf=y_true.request(); py::array_t<double> p=sigmoid(y_pred); auto p_buf=p.request();
    py::array_t<double> result(y_true_buf.size); auto res_buf=result.request();
    double *y_true_ptr=static_cast<double*>(y_true_buf.ptr), *p_ptr=static_cast<double*>(p_buf.ptr), *res_ptr=static_cast<double*>(res_buf.ptr);
    #pragma omp parallel for
    for (ssize_t i=0; i<y_true_buf.size; ++i) res_ptr[i] = p_ptr[i] - y_true_ptr[i];
    std::vector<ssize_t> shape(y_true.shape(), y_true.shape() + y_true.ndim());
    return result.reshape(shape);
}

py::array_t<double> hessian(py::array_t<double> y_true, py::array_t<double> y_pred) {
    py::array_t<double> p=sigmoid(y_pred); auto p_buf=p.request(); py::array_t<double> result(p_buf.size);
    auto res_buf = result.request(); double *p_ptr=static_cast<double*>(p_buf.ptr), *res_ptr=static_cast<double*>(res_buf.ptr);
    #pragma omp parallel for
    for (ssize_t i=0; i<p_buf.size; ++i) res_ptr[i] = std::max(p_ptr[i] * (1.0 - p_ptr[i]), 1e-7);
    std::vector<ssize_t> shape(p.shape(), p.shape() + p.ndim());
    return result.reshape(shape);
}

double init_score(py::array_t<double> y_true) {
    auto buf = y_true.request(); if (buf.size == 0) return 0.0;
    double sum = std::accumulate(static_cast<double*>(buf.ptr), static_cast<double*>(buf.ptr) + buf.size, 0.0);
    double p = std::max(1e-7, std::min(1.0 - 1e-7, sum / buf.size));
    return std::log(p / (1.0 - p));
}

// Helper for Shannon Entropy
double calculate_shannon_entropy(double count0, double count1) {
    double total = count0 + count1;
    if (total < 1e-9 || count0 < 1e-9 || count1 < 1e-9) return 0.0;
    double p0 = count0 / total, p1 = count1 / total;
    return -p0 * std::log2(p0) - p1 * std::log2(p1);
}

// Result Structure
struct HistSplitResult {
    double best_gain = -std::numeric_limits<double>::infinity();
    int best_bin_idx = 0;
};

// Histogram Structure
struct Histogram {
    std::vector<double> sum_gradients, sum_hessians, sum_y_true, counts;
};

// Split Finding Function
HistSplitResult find_best_split_shannon(
    py::array_t<int> x_binned, py::array_t<double> y, py::array_t<double> grad, py::array_t<double> hess,
    int n_bins, int min_samples_split, double min_child_weight, double reg_lambda,
    double gamma, double mi_weight, int depth)
{
    auto x_buf=x_binned.request(), y_buf=y.request(), g_buf=grad.request(), h_buf=hess.request();
    ssize_t n_samples = x_buf.shape[0];
    int *x_ptr=static_cast<int*>(x_buf.ptr); double *y_ptr=static_cast<double*>(y_buf.ptr);
    double *g_ptr=static_cast<double*>(g_buf.ptr), *h_ptr=static_cast<double*>(h_buf.ptr);

    // Thread-local histograms
    int n_threads = omp_get_max_threads();
    std::vector<Histogram> private_hists;
    for (int i = 0; i < n_threads; ++i) {
        private_hists.push_back({std::vector<double>(n_bins,0.0), std::vector<double>(n_bins,0.0),
                                std::vector<double>(n_bins,0.0), std::vector<double>(n_bins,0.0)});
    }

    #pragma omp parallel for
    for(ssize_t i=0; i<n_samples; ++i) {
        int thread_id = omp_get_thread_num();
        int bin = x_ptr[i];
        if (bin>=0 && bin<n_bins) {
            private_hists[thread_id].sum_gradients[bin]+=g_ptr[i];
            private_hists[thread_id].sum_hessians[bin]+=h_ptr[i];
            private_hists[thread_id].sum_y_true[bin]+=y_ptr[i];
            private_hists[thread_id].counts[bin]+=1.0;
        }
    }

    // Reduce histograms
    Histogram hist{std::vector<double>(n_bins,0.0), std::vector<double>(n_bins,0.0),
                   std::vector<double>(n_bins,0.0), std::vector<double>(n_bins,0.0)};
    for (int bin = 0; bin < n_bins; ++bin) {
        for (int i = 0; i < n_threads; ++i) {
            hist.sum_gradients[bin] += private_hists[i].sum_gradients[bin];
            hist.sum_hessians[bin] += private_hists[i].sum_hessians[bin];
            hist.sum_y_true[bin] += private_hists[i].sum_y_true[bin];
            hist.counts[bin] += private_hists[i].counts[bin];
        }
    }

    double G_total=std::accumulate(hist.sum_gradients.begin(),hist.sum_gradients.end(),0.0);
    double H_total=std::accumulate(hist.sum_hessians.begin(),hist.sum_hessians.end(),0.0);
    double y_total_sum=std::accumulate(hist.sum_y_true.begin(),hist.sum_y_true.end(),0.0);
    double n_total_samples=std::accumulate(hist.counts.begin(),hist.counts.end(),0.0);

    if (n_total_samples < min_samples_split) return HistSplitResult();

    double parent_entropy = calculate_shannon_entropy(n_total_samples - y_total_sum, y_total_sum);

    double GL=0.0, HL=0.0, y_left_sum=0.0, n_left_samples=0.0;
    HistSplitResult best_split;

    for (int i=0; i<n_bins-1; ++i) {
        GL+=hist.sum_gradients[i]; HL+=hist.sum_hessians[i];
        y_left_sum+=hist.sum_y_true[i]; n_left_samples+=hist.counts[i];

        if (HL < min_child_weight || n_left_samples < min_samples_split) continue;
        double GR=G_total-GL, HR=H_total-HL;
        double n_right_samples = n_total_samples - n_left_samples;
        if (HR < min_child_weight || n_right_samples < min_samples_split) continue;

        double newton_gain = 0.5*((GL*GL/(HL+reg_lambda))+(GR*GR/(HR+reg_lambda))-(G_total*G_total/(H_total+reg_lambda))) - gamma;

        double y_right_sum = y_total_sum - y_left_sum;
        double h_left = calculate_shannon_entropy(n_left_samples - y_left_sum, y_left_sum);
        double h_right = calculate_shannon_entropy(n_right_samples - y_right_sum, y_right_sum);
        double h_conditional = (n_left_samples / n_total_samples) * h_left + (n_right_samples / n_total_samples) * h_right;
        double mutual_info = parent_entropy - h_conditional;

        double adaptive_weight = mi_weight * std::exp(-0.1 * static_cast<double>(depth));
        double combined_gain = newton_gain + adaptive_weight * mutual_info;

        if (combined_gain > best_split.best_gain) {
            best_split.best_gain = combined_gain;
            best_split.best_bin_idx = i;
        }
    }
    return best_split;
}

// Module Definition
PYBIND11_MODULE(pknumpy_shannon_v2, m) {
    m.doc() = "PKNumpy-Shannon";

    // Export functions
    m.def("sigmoid", &sigmoid, "Sigmoid activation function");
    m.def("gradient", &gradient, "Gradient calculation");
    m.def("hessian", &hessian, "Hessian calculation");
    m.def("init_score", &init_score, "Initialize base score");
    m.def("find_best_split_shannon", &find_best_split_shannon, "Find best split using Shannon MI");

    // Export result class
    py::class_<HistSplitResult>(m, "HistSplitResult")
        .def(py::init<>())
        .def_readonly("best_gain", &HistSplitResult::best_gain)
        .def_readonly("best_bin_idx", &HistSplitResult::best_bin_idx);
}

Overwriting pknumpy_shannon_v2.cpp


In [4]:
# Cell 3: Fixed Manual Compilation and Import
import sys
import pybind11
import importlib.util
import os
import subprocess

# Clear any existing module to avoid registration conflicts
if 'pknp_shannon' in globals():
    del pknp_shannon

# Remove any existing compiled modules
for file in os.listdir('/content/'):
    if file.startswith('pknumpy_shannon') and (file.endswith('.so') or '.cpython-' in file):
        try:
            os.remove(f'/content/{file}')
            print(f"Removed existing module: {file}")
        except:
            pass

pknp_shannon = None
py_include = f"-I/usr/include/python{sys.version_info.major}.{sys.version_info.minor}"
pybind_include = f"-I{pybind11.get_include()}"

try:
    extension_suffix = subprocess.check_output(['python3-config', '--extension-suffix'], text=True).strip()
    module_name = f"pknumpy_shannon_v2{extension_suffix}"  # Use different name to avoid conflicts

    # Compile with proper flags
    compile_command = f"g++ -O3 -shared -fPIC -std=c++17 -fopenmp {py_include} {pybind_include} pknumpy_shannon.cpp -o {module_name}"

    print("Compiling Optimized Shannon backend with multi-threading...")
    compile_process = subprocess.run(compile_command, shell=True, check=True, capture_output=True, text=True)

    print("Compilation successful.")

    # Import the module
    module_path = f"/content/{module_name}"
    spec = importlib.util.spec_from_file_location("pknumpy_shannon", module_path)
    pknp_shannon = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(pknp_shannon)

    # Verify the module has the required functions
    required_functions = ['sigmoid', 'gradient', 'hessian', 'init_score', 'find_best_split_shannon']
    missing_functions = [func for func in required_functions if not hasattr(pknp_shannon, func)]

    if missing_functions:
        print(f"ERROR: Module missing functions: {missing_functions}")
        print("Available attributes:", [attr for attr in dir(pknp_shannon) if not attr.startswith('_')])
        pknp_shannon = None
    else:
        print("Successfully imported Optimized Shannon C++ module as `pknp_shannon`.")
        print("All required functions are available.")

except (FileNotFoundError, subprocess.CalledProcessError, ImportError) as e:
    print(f"ERROR compiling/importing backend: {e}")
    if hasattr(e, 'stderr') and e.stderr:
        print("Compilation stderr:", e.stderr)
    if hasattr(e, 'stdout') and e.stdout:
        print("Compilation stdout:", e.stdout)
    pknp_shannon = None

Compiling Optimized Shannon backend with multi-threading...
ERROR compiling/importing backend: Command 'g++ -O3 -shared -fPIC -std=c++17 -fopenmp -I/usr/include/python3.12 -I/usr/local/lib/python3.12/dist-packages/pybind11/include pknumpy_shannon.cpp -o pknumpy_shannon_v2.cpython-310-x86_64-linux-gnu.so' returned non-zero exit status 1.
Compilation stderr: cc1plus: fatal error: pknumpy_shannon.cpp: No such file or directory
compilation terminated.



In [5]:
# Cell 4: The Final PKBoostShannon Python Class
# This cell defines the final, clean Python classes that use the

import time
import numpy as np
from typing import Dict, List, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

class SmartHistogramBuilder:
    def __init__(self, max_bins: int=32): self.max_bins=max_bins; self.bin_edges_=None; self.n_bins_per_feature_=None
    def fit(self, X:np.ndarray):
        n_features=X.shape[1]; self.bin_edges_=[]; self.n_bins_per_feature_=[]
        for i in range(n_features):
            unique_vals=np.unique(X[:, i]);
            if len(unique_vals) <= self.max_bins: edges=unique_vals
            else: edges=np.unique(np.quantile(X[:, i], np.linspace(0, 1, self.max_bins + 1)))
            self.bin_edges_.append(edges); self.n_bins_per_feature_.append(len(edges))
        return self
    def transform(self, X:np.ndarray) -> np.ndarray:
        X_binned=np.zeros_like(X, dtype=np.int32)
        for i in range(X.shape[1]):
            X_binned[:, i] = np.clip(np.searchsorted(self.bin_edges_[i], X[:, i], side='right') - 1, 0, self.n_bins_per_feature_[i] - 2)
        return X_binned

class CppShannonLoss:
    @staticmethod
    def sigmoid(x): return pknp_shannon.sigmoid(x)
    def gradient(self, y, p): return pknp_shannon.gradient(y, p)
    def hessian(self, y, p): return pknp_shannon.hessian(y, p)
    def init_score(self, y): return pknp_shannon.init_score(y)

class SimpleTreeShannon:
    def __init__(self, max_depth:int=6): self.max_depth=max_depth; self.nodes={}
    def fit(self, X_binned, y, grad, hess, params):
        self._build_tree(X_binned, y, grad, hess, 0, 0, params)
    def _build_tree(self, X_binned, y, grad, hess, node_id, depth, params):
        n_samples, G, H = X_binned.shape[0], np.sum(grad), np.sum(hess)
        if depth>=self.max_depth or n_samples<params['min_samples_split'] or H<params['min_child_weight']:
            self.nodes[node_id]={'is_leaf':True, 'value': -G / (H + params['reg_lambda'])}; return
        best_gain, best_feature, best_threshold_bin = -np.inf, -1, 0
        for feat_idx in range(X_binned.shape[1]):
            result = pknp_shannon.find_best_split_shannon(
                np.ascontiguousarray(X_binned[:, feat_idx], dtype=np.int32), np.ascontiguousarray(y),
                np.ascontiguousarray(grad), np.ascontiguousarray(hess),
                params['n_bins_per_feature'][feat_idx], params['min_samples_split'], params['min_child_weight'],
                params['reg_lambda'], params['gamma'], params['mi_weight'], depth
            )
            gain, threshold_bin = result.best_gain, result.best_bin_idx
            if gain > best_gain: best_gain, best_feature, best_threshold_bin = gain, feat_idx, threshold_bin
        if best_gain <= 1e-6:
            self.nodes[node_id]={'is_leaf':True, 'value':-G/(H+params['reg_lambda'])}; return
        self.nodes[node_id]={'is_leaf':False,'feature':best_feature,'threshold':best_threshold_bin,'left_child':2*node_id+1,'right_child':2*node_id+2}
        left_mask = X_binned[:, best_feature] <= best_threshold_bin
        self._build_tree(X_binned[left_mask], y[left_mask], grad[left_mask], hess[left_mask], 2*node_id+1, depth+1, params)
        self._build_tree(X_binned[~left_mask], y[~left_mask], grad[~left_mask], hess[~left_mask], 2*node_id+2, depth+1, params)
    def predict(self, X_binned:np.ndarray)->np.ndarray:
        preds = np.zeros(X_binned.shape[0])
        for i in range(X_binned.shape[0]):
            node_id=0
            while node_id in self.nodes:
                node=self.nodes[node_id]
                if node['is_leaf']: preds[i]=node['value']; break
                node_id = node['left_child'] if X_binned[i, node['feature']] <= node['threshold'] else node['right_child']
        return preds

class PKBoostShannon:
    def __init__(self, n_estimators=500, learning_rate=0.1, max_depth=6, min_samples_split=20,
                 min_child_weight=1.0, reg_lambda=1.0, gamma=0.0, subsample=0.8,
                 colsample_bytree=0.8, early_stopping_rounds=50, histogram_bins=32,
                 random_state=None, mi_weight=0.1):
        params=locals(); del params["self"];
        for k, v in params.items(): setattr(self, k, v)
        self.trees:List[SimpleTreeShannon]=[]; self.base_score,self.best_iteration,self.best_score=0.0,0,-np.inf
        self.fitted=False; self.loss_fn=CppShannonLoss()
        if random_state is not None: np.random.seed(random_state)
    def fit(self, X, y, eval_set=None, verbose=True):
        start_time=time.time(); X, y=np.asarray(X), np.asarray(y)
        n_samples, n_features = X.shape; self.base_score=self.loss_fn.init_score(y)
        train_preds=np.full(n_samples, self.base_score)
        self.histogram_builder = SmartHistogramBuilder(self.histogram_bins).fit(X)
        X_processed = self.histogram_builder.transform(X)
        if eval_set:
            X_val, y_val=np.asarray(eval_set[0]), np.asarray(eval_set[1])
            X_val_processed = self.histogram_builder.transform(X_val)
            val_preds = np.full(len(y_val), self.base_score)
        if verbose: print(f"PKBoost Training Started on {n_samples} samples")
        for iteration in range(self.n_estimators):
            s_idxs=np.random.choice(n_samples, int(self.subsample*n_samples), replace=False)
            f_idxs=np.random.choice(n_features, max(1, int(self.colsample_bytree*n_features)), replace=False)
            grad, hess = self.loss_fn.gradient(y[s_idxs], train_preds[s_idxs]), self.loss_fn.hessian(y[s_idxs], train_preds[s_idxs])

            tree = SimpleTreeShannon(self.max_depth)
            tree_params = {p: getattr(self,p) for p in ["min_samples_split", "min_child_weight", "reg_lambda", "gamma", "mi_weight"]}
            tree_params['n_bins_per_feature'] = [self.histogram_builder.n_bins_per_feature_[i] for i in f_idxs]

            tree.fit(X_processed[s_idxs][:, f_idxs], y[s_idxs], grad, hess, tree_params)
            tree.feature_indices=f_idxs; self.trees.append(tree)
            train_preds += self.learning_rate * tree.predict(X_processed[:, f_idxs])
            if eval_set:
                val_preds += self.learning_rate * tree.predict(X_val_processed[:, f_idxs])
                val_roc = roc_auc_score(y_val, self.loss_fn.sigmoid(val_preds))
                if val_roc > self.best_score: self.best_score, self.best_iteration=val_roc, iteration
                elif iteration-self.best_iteration>=self.early_stopping_rounds:
                    if verbose: print(f"Early stopping at iter {iteration+1}"); break
            else: self.best_iteration = iteration
            if verbose and (iteration+1)%50==0 and eval_set: print(f"[{iteration+1:3d}] Val ROC: {val_roc:.4f}")
        self.fitted=True
        if verbose: print(f"Training completed in {time.time()-start_time:.2f}s. Best Val ROC: {self.best_score:.6f}")
        return self
    def predict_proba(self, X:np.ndarray)->np.ndarray:
        if not self.fitted: raise ValueError("Model not fitted")
        X_proc=self.histogram_builder.transform(np.asarray(X))
        preds=np.full(X_proc.shape[0], self.base_score)
        for tree in self.trees[:self.best_iteration+1]:
            preds += self.learning_rate * tree.predict(X_proc[:, tree.feature_indices])
        return self.loss_fn.sigmoid(preds)
print("Python classes for PkBoost defined successfully.")

Python classes for PkBoost defined successfully.


In [ ]:
# Final Cell: Head-to-Head Benchmark vs. LightGBM
# -----------------------------------------------
# This cell installs lightgbm and then runs a direct comparison
# between your optimized PKBoostShannon model and LightGBM on the
# prepared Credit Card Fraud dataset.

import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import time

# --- Ensure lightgbm is installed ---
!pip install lightgbm -q

# Check if data is loaded and the C++ module is ready
if 'data_loaded_successfully' in locals() and data_loaded_successfully and 'pknp_shannon' in locals():
    print("\n" + "="*80)
    print("FINAL BENCHMARK: PKBoost vs. LightGBM")
    print("="*80)

    # --- 1. Your PKBoost (Optimized Shannon) Model ---
    print("\n1. PkBoost MODEL...")
    pkb_model = PKBoostShannon(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        early_stopping_rounds=30,
        random_state=42,
        mi_weight=0.5
    )
    pkb_start_time = time.time()
    pkb_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
    pkb_time = time.time() - pkb_start_time
    pkb_roc = roc_auc_score(y_test, pkb_model.predict_proba(X_test))
    print(f"PKBoost training complete.")

    # --- 2. LightGBM Model ---
    print("\n2. TRAINING LIGHTGBM MODEL...")
    lgbm_model = lgb.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        random_state=42,
        n_jobs=-1 # Use all available cores
    )
    lgbm_start_time = time.time()
    # Use a callback for early stopping in LightGBM
    lgbm_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='auc',
        callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)]
    )
    lgbm_time = time.time() - lgbm_start_time
    lgbm_roc = roc_auc_score(y_test, lgbm_model.predict_proba(X_test)[:, 1])
    print(f"LightGBM training complete.")

    # --- 3. Logistic Regression Baseline ---
    lr = LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42, class_weight='balanced')
    lr.fit(X_train,y_train)
    lr_roc = roc_auc_score(y_test, lr.predict_proba(X_test)[:, 1])

    # --- Final Comparison ---
    print("\n" + "="*80)
    print("FINAL PERFORMANCE COMPARISON")
    print("="*80)
    print(f"{'Model':<40} {'Test ROC-AUC':<15} {'Time (s)':<10}")
    print("-" * 70)
    print(f"{'Logistic Regression':<40} {lr_roc:<15.6f} {'N/A'}")
    print(f"{'PKBoost ':<40} {pkb_roc:<15.6f} {pkb_time:<10.2f}")
    print(f"{'LightGBM ':<40} {lgbm_roc:<15.6f} {lgbm_time:<10.2f}")
    print("="*70)
else:
    print("\nSkipping final benchmark")


FINAL BENCHMARK: PKBoost vs. LightGBM

1. TRAINING YOUR PKBOOST (OPTIMIZED SHANNON) MODEL...
  -> PKBoost training complete.

2. TRAINING LIGHTGBM MODEL...
[LightGBM] [Info] Number of positive: 275, number of negative: 28000
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007624 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 28275, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.009726 -> initscore=-4.623189
[LightGBM] [Info] Start training from score -4.623189
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv